In [ ]:
!pip install tensorflow

In [ ]:
import os
import numpy as np
import xarray as xr
from glob import glob
from tqdm import tqdm

# Parameters
PATCH_SIZE = 12
BANDS = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 
         'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
MAX_PATCHES_PER_FILE = 100  # You can adjust based on memory

def extract_patches_from_file(file_path, label, max_patches):
    ds = xr.open_dataset(file_path)
    
    # Keep only required bands
    ds = ds[[b for b in BANDS if b in ds.data_vars]]
    
    if "t" in ds.dims:
        ds = ds.isel(t=0)
    
    da = ds.to_array().transpose("y", "x", "variable")  # shape: (H, W, 12)
    data = da.values.astype(np.float32)
    
    # Pad if necessary to ensure full patches
    h, w, c = data.shape
    if h < PATCH_SIZE or w < PATCH_SIZE:
        return [], []

    patches = []
    labels = []
    
    # Randomly sample patch locations
    for _ in range(max_patches):
        y = np.random.randint(0, h - PATCH_SIZE + 1)
        x = np.random.randint(0, w - PATCH_SIZE + 1)
        patch = data[y:y+PATCH_SIZE, x:x+PATCH_SIZE, :]

        if not np.isnan(patch).any():
            patches.append(patch)
            labels.append(label)

    return patches, labels


def build_datacube_dataset(folder_path):
    all_patches = []
    all_labels = []

    files = sorted(glob(os.path.join(folder_path, "*.nc")))
    print(f"Found {len(files)} files.")

    for file_path in tqdm(files):
        label = 1 if "after" in os.path.basename(file_path).lower() else 0
        patches, labels = extract_patches_from_file(file_path, label, MAX_PATCHES_PER_FILE)
        all_patches.extend(patches)
        all_labels.extend(labels)

    X = np.array(all_patches)  # shape: (N, 12, 12, 12)
    y = np.array(all_labels)   # shape: (N,)
    return X, y


In [ ]:
X, y = build_datacube_dataset("datacubes_2024")
print("X shape:", X.shape)  # (samples, 12, 12, 12)
print("y shape:", y.shape)  # (samples,)

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Assuming X shape: (N, 12, 12, 12), y shape: (N,)
# Reshape if needed: TensorFlow prefers channels-last: (samples, height, width, depth)
X = X.astype("float32")
y = y.astype("int32")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Create TensorFlow datasets
BATCH_SIZE = 32

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)) \
    .shuffle(buffer_size=1000) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE)


<h1>2d cnn</h1>

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(12, 12, 12)),  # (H, W, Bands)
    
    layers.Conv2D(32, kernel_size=3, activation='relu'),
    layers.MaxPooling2D(pool_size=2),
    
    layers.Conv2D(64, kernel_size=3, activation='relu'),
    layers.Flatten(),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10  # You can increase this
)


In [ ]:
import numpy as np
import xarray as xr
import tensorflow as tf
import rasterio
from rasterio.transform import from_origin
import os

# SETTINGS
PATCH_SIZE = 12
BANDS = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06',
         'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']

def classify_blocks_with_model(nc_file_path, model, output_tiff_path):
    # Load the dataset
    ds = xr.open_dataset(nc_file_path)
    ds = ds[[b for b in BANDS if b in ds.data_vars]]
    if "t" in ds.dims:
        ds = ds.isel(t=0)

    # Create 3D array (H, W, Bands)
    da = ds.to_array().transpose("y", "x", "variable")
    data = da.values.astype(np.float32)

    height, width, bands = data.shape
    if height < PATCH_SIZE or width < PATCH_SIZE:
        raise ValueError("Image too small for one patch.")

    # Compute how many full patches fit
    n_y = height // PATCH_SIZE
    n_x = width // PATCH_SIZE

    # Create block grid for output
    prediction_map = np.full((n_y, n_x), fill_value=-1, dtype=np.int8)

    blocks = []
    coords = []

    for i in range(n_y):
        for j in range(n_x):
            y = i * PATCH_SIZE
            x = j * PATCH_SIZE
            patch = data[y:y+PATCH_SIZE, x:x+PATCH_SIZE, :]

            if not np.isnan(patch).any():
                patch = patch[..., np.newaxis]  # (12, 12, 12, 1)
                blocks.append(patch)
                coords.append((i, j))

    if not blocks:
        raise ValueError("No valid patches found for classification.")

    # Convert to tensor
    X_blocks = np.stack(blocks)  # (N, 12, 12, 12, 1)
    predictions = model.predict(X_blocks, batch_size=32)
    labels = (predictions.flatten() > 0.5).astype(np.uint8)

    # Fill prediction map
    for (i, j), label in zip(coords, labels):
        prediction_map[i, j] = label

    # Scale up to original resolution for GeoTIFF (nearest-neighbor repeat)
    full_map = np.kron(prediction_map, np.ones((PATCH_SIZE, PATCH_SIZE), dtype=np.uint8))

    # Build transform and save GeoTIFF
    transform = from_origin(
        float(ds.x[0]), float(ds.y[0]),
        float(ds.x[1] - ds.x[0]),
        float(ds.y[0] - ds.y[1])
    )

    with rasterio.open(
        output_tiff_path,
        'w',
        driver='GTiff',
        height=full_map.shape[0],
        width=full_map.shape[1],
        count=1,
        dtype=rasterio.uint8,
        crs=ds.rio.crs if hasattr(ds, 'rio') else 'EPSG:4326',
        transform=transform,
    ) as dst:
        dst.write(full_map, 1)

    print(f"✅ Saved classified TIFF to: {output_tiff_path}")


In [ ]:
# Load trained model
#model = tf.keras.models.load_model("cnn_model_3d.h5")

# Classify patches in .nc and write TIFF
classify_blocks_with_model("datacubes_2024/fire_226116_before.nc", model, "classified_2dblocks.tif")


<h1>3D cnn</h1>

In [ ]:
from tensorflow.keras import layers, models

model_3d = models.Sequential([
    layers.Input(shape=(12, 12, 12, 1)),  # (depth, height, width, channels)
    
    layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    
    layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')  # Binary classification: burned vs non-burned
])

model_3d.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_3d.summary()


In [ ]:
# Reshape from (N, 12, 12, 12) → (N, 12, 12, 12, 1)
X_train_3d = X_train[..., np.newaxis]
X_test_3d = X_test[..., np.newaxis]

train_ds_3d = tf.data.Dataset.from_tensor_slices((X_train_3d, y_train)) \
    .shuffle(1000) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE)

test_ds_3d = tf.data.Dataset.from_tensor_slices((X_test_3d, y_test)) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE)


In [ ]:
history = model_3d.fit(
    train_ds_3d,
    validation_data=test_ds_3d,
    epochs=10
)


In [ ]:
classify_blocks_with_model("datacubes_2024/fire_226116_after.nc", model_3d, "classified_3dblocks_Fire.tif")


<h2> attention based</h2>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_attention_model(input_shape=(12, 12, 12)):
    input_tensor = layers.Input(shape=input_shape)  # (H, W, Bands)

    # Reshape to (tokens, features): 144 tokens, each 12D (spectral)
    x = layers.Reshape((144, 12))(input_tensor)  # (batch, 144, 12)

    # Positional embedding (learnable)
    pos_embed = layers.Embedding(input_dim=144, output_dim=12)
    positions = tf.range(start=0, limit=144, delta=1)
    x += pos_embed(positions)

    # Attention block
    x = layers.LayerNormalization()(x)
    x = layers.MultiHeadAttention(num_heads=4, key_dim=12)(x, x)
    x = layers.LayerNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)

    # Classification head
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=input_tensor, outputs=output)
    return model


In [ ]:
model = build_attention_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train
model.fit(train_ds, validation_data=test_ds, epochs=10)


In [ ]:
classify_blocks_with_model("datacubes_2024/fire_226116_after.nc", model, "classified_attentio_Fire.tif")
